Change log:

* 5/17/2025: Added notes up to "Why B-Trees Are The Default choice". Will continue adding hand-written notes from video later.
* 5/18/2025: Finished adding hand-written notes on the other types of indexes and when to use them.

# Database Indexing

## How Database Indexes Work

* data in a database is written to disk as a collection of files
    - think of it like a notebook where you write line by line when a new thought comes in

### Physical Storage and Access Patterns

* when data lives on disk (SSDs nowadays), we can only process that data when we bring it into memory
    - every query has to load data from disk => RAM
* without an index, you have to scan through every page of data one by one
    - loading each page into memory and scanning for the item you're looking for
    - e.g. like looking through every page of a book to find a specific word
* but an index gives us a structured path to the data we need
    - it can tell us which pages contain the data we're looking for
    - this is like using the Table of Contents to jump to the relevant pages

### The Cost of Indexing

* indexes are not free though
* they require additonal disk space since they're a data structure
    - and they might even occupy as much space as the original data
* write performance takes a hit
    - inserting new rows/updating existing rows will not only update the main table but also every index on it
    - multiple indexes = multipe disk writes for a single write operation
* __when do indexes hurt more than help?__:
    - when a table has frequent writes but infrequent reads
        - e.g. logging table where we insert new records but rarely query old ones
    - or when the table is small with just a few hundred rows
        - cost of maintaining an index might be more than cost of a simple sequentialscan

## Types of Indexes:

### B-Tree Indexes:

* most common type of index

#### The Structure of B-Trees:

* self-balancing tree
    - maintains __sorted data__
    - allows efficient insertions, deletions, and searches
    - can have multiple children (hundreds in practice)
    - each node contains an ordered array of keys and pointers, structured to minimize disk reads
* every node in a B-tree follows strict rules:
    - all leaf nodes must be at the same depth
    - each node can contain between m/2 and m keys (where m is the order of the tree)
        * order(m) = max # of children a node can have
    - a node with k keys must have exactly k+1 children
    - keys within a node are kept in sorted order
* each node fits in a single disk page
    - typicall 8KB
    - e.g. when PostgreSQL needs to find a record with id=350, it might only need to read 2-3 pages from disk: root node => internal node => leaf node

#### Real-World Examples

* PostgreSQL uses B-trees for almost everything: primary keys, unique constraints, and most regular indexs
    ```
    CREATE TABLE users {
        id SERIAL PRIMARY KEY,
        email VARCHAR(255) UNIQUE
    }
    ```
    - this automatically creates 2 B-tree indexes: one for the primary key and one for the unique email constraint
    - these B-trees maintain sorted order
* when you create an index in MongoDB: db.users.createIndex({ "email": 1 });
    - you create a B-tree that maps email values to document locations

#### Why B-trees Are the Default Choice:

* they excel at everything databases need
* B-trees are a safe bet to use for indexes in interviews
1. maintain sorted order, making range queries and ORDER BY operations efficient
2. self-balancing, ensuring predictable performance even as data grows
3. minimize disk I/O by matching their structure to how databases store data
4. handle both equality searches (email='X') and range searches (age > 25) equally well
5. remain balanced even with random inserts and deletes, avoiding the performance cliffs you might see with simpler tree structures

## Hash Index:

* uses a hash map
* pass data into hash function to get a key
    - key will grab value, which is a page where our item is
* rarely used in production databases
    - they offer O(1) lookups but B-trees not only perform just as well for exact matches but also for range queries and sorting too
* hash indexes used mostly for in-memory stores like Redis
    - where disk-I/O patterns not relevant
    - useful for caching

## Geospatial Index:

* search regions using latitude/longitude
    - e.g. yelp, find nearby friends
* B-trees not good here b/c B-trees good for __1-Dimensional data__
    - in order to grab data for a region using latitude and longitutde, B-trees must query:
        1. all data for latitutde
        2. all data for longitude
        3. merge them together
    - so you get huge slices of data in both dimensions and have to perform an expensive merge
* for Geospatial indexes, we have special indexes/algorithms:
    1. Geohashing
    2. Quad Trees
    3. R-Trees

### Geohashing (unconditional recursive split into hash strings)

* here's how it works:
    1. take map of world
    2. split into 4 parts
    3. label each part as hash strings
    4. then for each part recursively split it up further
* for every split, we get increasing levels of precision
* when you convert latitutde/longitude into these hash strings, we find that nearby areas share the same __prefix__
    - e.g. the Bay Area might have a hash string of 123, but San Francisco has a hash string of 1234 while Daly City has a hash string of 1235
    - regardless, they both share the same prefix: 123
* we can then build a B-tree on top of these hashes and get ~O(1) lookups
    - normally use base-32 encoding for Geohash
    - e.g. LA = 9qh16

### Quad Trees (conditional recursive split into tree)

* similar to Geohash in which we split world into parts recursively
    - but we split the world into a tree structure, not hash strings
    - also, we only split parts with __high density__
        * density determined by a k-value
        * if # of objects in a part > k, then split it further
        * and that split is now a __child node__ of its parent node
* just traverse down quad tree for lookups

### R-Tree:

* derived from Quad Trees
    - however, has dynamic splitting/clustering to group nearby items
    - still represented as a tree but not splitting exactly by 4s

### Overview of Geospatial Indexes:

* Geohashing = very popular
    - used in Redis, production databases
    - can use database indexes on top of it like B-trees
* Quad Trees: foundational but not used in production
* R-Tree: used in production like PostGIS

## Inverted Index

* used for text search
    - B-trees only good for finding prefixes
    - but text can appear in ANY position so can't use it
* how does it work?
    - create a hash map
    - key = words/tokens
    - value = list of documents they appear in
    - so search for text and get list of docs

## When to Use Indexes?

* if low data access or low data => full table scan, no index needed
* if table size > 10k rows => use index
* What type of data?
    - full text search = Inverted Index
    - location data => Geospatial Index
    - exact in-memory matches => hash index/B-tree
    - everything else (default): B-trees